# POC show Ibis CLL within Kedro pipeline DAG

In [3]:
from kedro.runner.sequential_runner import SequentialRunner
from kedro.io.data_catalog import DataCatalog
from jaffle_shop.utilities.graph import (
    kedro_node_ibis_plan, 
    kedro_pipeline_to_graph, 
    nx_graph_to_pyvis,
    get_lineage_for_kedro_node
)

Retrieve pipelines object (automatically in scope via `kedro jupyter notebook`)

In [4]:
default_pipeline = pipelines['__default__']
default_pipeline


Pipeline([
Node(rename_customers, 'raw_customers', 'stg_customers', None),
Node(rename_orders, 'raw_orders', 'stg_orders', None),
Node(rename_payments, 'raw_payments', 'stg_payments', None),
Node(process_customers, ['stg_customers', 'stg_orders', 'stg_payments'], 'customers', None),
Node(process_orders, ['stg_orders', 'stg_payments', 'params:payment_methods'], ['orders', 'temp_orders'], None)
])

## Render Kedro Pipeline (equivilent to Kedro-Viz)

In [5]:
nx_pipeline_graph = kedro_pipeline_to_graph(default_pipeline, catalog)
print(nx_pipeline_graph, type(nx_pipeline_graph))
nx_graph_to_pyvis(nx_pipeline_graph).show('../data/03_primary/kedro_pipeline.html')

DiGraph with 15 nodes and 15 edges <class 'networkx.classes.digraph.DiGraph'>
../data/03_primary/kedro_pipeline.html


<IPython.lib.display.IFrame object at 0x149a9c1f0>

## Retrieve Kedro Node

In [6]:
n = default_pipeline.nodes[2]
n

Node(rename_payments, 'raw_payments', 'stg_payments', None)

In [7]:
import inspect
print(inspect.getsource(n.func))

def rename_payments(source: ir.Table) -> ir.Table:
    # `amount` is currently stored in cents, so we convert it to dollars
    amount_in_dollars = (source.amount / 100).name("amount")

    renamed = source[
        source.id.name("payment_id"),
        "order_id",
        "payment_method",
        amount_in_dollars,
    ]
    return renamed



## Show Ibis CLL within Kedro node

In [8]:
nx_node_graph = kedro_node_ibis_plan(n,catalog)
nx_graph_to_pyvis(nx_node_graph).show('../data/03_primary/kedro_node.html')

../data/03_primary/kedro_node.html


<IPython.lib.display.IFrame object at 0x14f3ffd90>

## Combine Ibis CLL with wider Kedro DAG

In [9]:
nx_graph_to_pyvis(
    get_lineage_for_kedro_node(
        default_pipeline,
        default_pipeline.nodes[2],
        catalog
    )
).show('graph.html')

graph.html


<IPython.lib.display.IFrame object at 0x129f0a530>